In [2]:
#!pip install geopy
#!pip install folium
#!conda install -c conda-forge geocoder --yes


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import geocoder
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Libraries imported.')

Libraries imported.


# Part 1

In [3]:
#!conda install -c conda-forge wikipedia --yes
import wikipedia as wp
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header = 0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
df = df[df.Borough != 'Not assigned']
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [5]:
df = df.groupby(['Borough', 'Postcode'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
df.head()

,Borough,Postcode,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,North Toronto West
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


In [6]:
df.shape

(103, 3)

# Part 2

Get Latitude and Longitude according to the Postcodes using the geocoder package

In [7]:
lat_list = []
lng_list = []
post = []

for i in range(df.shape[0]):
    postcode = df['Postcode'].iloc[i]
    address = postcode+', Toronto, Ontario'
    g = geocoder.arcgis(address)
    post.append(postcode)
    lat_list.append(g.latlng[0])
    lng_list.append(g.latlng[1])

In [10]:
latlng_df = pd.DataFrame(
    {'Postcode': post,
     'Latitude': lat_list,
     'Longitude': lng_list
    })

tor_df = pd.merge(df, latlng_df, on='Postcode')
tor_df = tor_df[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
tor_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133
1,M4P,Central Toronto,Davisville North,43.712755,-79.388514
2,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
3,M4S,Central Toronto,Davisville,43.703395,-79.385964
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.690685,-79.382946
5,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686074,-79.402265
6,M5N,Central Toronto,Roselawn,43.711941,-79.419120
7,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.694785,-79.414405
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.674840,-79.403768
9,M4W,Downtown Toronto,Rosedale,43.682205,-79.377945
